# SCC-DFTB Dataset generation

Input: DFT_DB.db - ASE database with coordinates, energies and forces, and Slater-Koster tables with dummy repulsive potential. 

Output: corresponding DFTB_DB.db with E<sup>BL</sup>+E<sup>SCC</sup> for the structures in DFT_DB.db


In this book, the calculations are done in serial. The computations can be run in parallel, which speeds up this step significantly.  

In [ ]:
from ase.io import read,write
from ase.io.vasp import read_vasp_out
from ase.atoms import *
from ase.calculators.dftb import Dftb
from ase.calculators.vasp import Vasp
from ase.optimize import *
import ase.db
import sys, os
import numpy as np
from tqdm import tqdm 
from ase.db import connect
os.environ['DFTB_COMMAND'] = 'dftb+'  # link to your DFTB binary. Modify if needed. 
base_dir=os.getcwd()
os.chdir(base_dir) 
if not os.path.isdir("RUN_SK/"):
    os.mkdir("RUN_SK/")

#os.chdir(base_dir)
print('You are here:', base_dir)

In [ ]:
# set up model for the DFTB+ computations using ASE 

def setup_model(elements,spin,charge,**kwargs):
    print('setting up calculator')
    os.environ["DFTB_PREFIX"] = base_dir+"/SKF/"
    DFTB_calc=Dftb(Hamiltonian_SCC='Yes',Hamiltonian_ShellResolvedSCC = 'Yes',Hamiltonian_SCCTolerance = '1.0E-006',Hamiltonian_Mixer= 'Anderson{',Hamiltonian_Mixer_MixingParameter='0.03',Hamiltonian_MaxSCCIterations = '150',Hamiltonian_Filling = 'MethfesselPaxton{',Hamiltonian_Filling_Temperature = '0.001583407672623195',Hamiltonian_KPointsAndWeights = 'SupercellFolding{1 0 0 0 1 0 0 0 1 0.0 0.0 0.0}',) 
    if spin>0:                  
        DFTB_calc.set(Hamiltonian_SpinPolarisation = 'Colinear{') 
        DFTB_calc.set(Hamiltonian_SpinPolarisation_UnpairedElectrons=str(spin))
        DFTB_calc.set(Hamiltonian_SpinConstants_ ='')
        for i in elements: 
            print(i)
            spinparam=get_spindata(i)
            string=f'Hamiltonian_SpinConstants_{i}'  
            print(string)
            DFTB_calc.set(**{string:spinparam})  
    if charge>0: 
        DFTB_calc.set(Hamiltonian_Charge =str(charge))
    calc=DFTB_calc
    return calc
           
def get_spindata(element): 
    if element=='Li':
        spinparam='{-0.019  -0.016  -0.016 -0.027}'
    if element=='C':
        spinparam='{-0.031 -0.025 -0.025 -0.023}'
    if element=='O':
        spinparam='{-0.035 -0.030 -0.030 -0.028}'
    if element=='H':
        spinparam='{-0.072}'
    if element=='P':
        spinparam='{-0.035 -0.030 -0.030 -0.0280}'   
    return spinparam 

def run_dftb(structure,SKF):
    os.chdir(base_dir+'/RUN_SK')
    w = structure.get_chemical_symbols()
    print('w is',w)
    charge=0
    spin=0
    calculator = setup_model(w,spin,charge)
    structure.calc=calculator
    structure.get_potential_energy()
    structure.get_forces()
    os.chdir(base_dir)
    return structure    

print('Models loaded')

In [ ]:
os.chdir(base_dir)

SKF=base_dir+"/SKF/"
dbname_DFT=base_dir+'/DFT_DB.db'
dbname_DFTB=base_dir+'/DFTB_DB.db'
db_dftb=ase.db.connect(dbname_DFTB)
db_dft=ase.db.connect(dbname_DFT)

for row in tqdm(db_dft.select()):
    structure=row.toatoms()
#    print(structure)
    try: 
        structure=run_dftb(structure,SKF)
        db_dft.update(row.id, key=row.id, DFTB=True)
#        print('DFT_DB updated')
        db_dftb.write(structure,key=row.id)
#        print("structure", row.key, "written to DFTB_db \"")
    except:
        print('Error')

print('Database',dbname_DFTB,'created from',dbname_DFT)